# Baseline BM25 sobre FiQA – TFM_QA_RAG

**Propósito:**  
Implementar y evaluar el sistema de recuperación clásico BM25 sobre el subset limpio de FiQA, servirá de baseline para posteriores comparativas.

**Archivos de entrada:**  
- ../data/subset_corpus_clean.csv  
- ../data/subset_queries_clean.csv  
- ../data/subset_qrels_clean.csv

---

## Índice

1. [Introducción y objetivos](#1-introducción-y-objetivos)
2. [Carga de datos y preparación](#2-carga-de-datos-y-preparación)
3. [Implementación de BM25](#3-implementación-de-bm25)
4. [Evaluación de resultados](#4-evaluación-de-resultados)
    - 4.1. Métricas agregadas (Precision@k, Recall@k, nDCG@k, MAP)
    - 4.2. Análisis por query
5. [Análisis cualitativo](#5-análisis-cualitativo)
    - 5.1. Ejemplos de queries fáciles/difíciles para BM25
    - 5.2. Limitaciones del enfoque clásico


---
## 1. Introducción y objetivos

El objetivo de este notebook es implementar y evaluar el rendimiento del sistema clásico de recuperación de información BM25 sobre el subset limpio del dataset FiQA (Financial Question Answering), preparado previamente en el análisis exploratorio.

El subset utilizado consta de:
- **300 queries**
- **3.000 documentos**
- **778 pares relevantes (qrels)**

Durante el análisis exploratorio se confirmó que:
- Todas las queries tienen al menos un documento relevante, asegurando evaluabilidad real.
- La mayoría de documentos extra funcionan como distractores reales, replicando un escenario de recuperación desafiante.
- No existen duplicados ni textos vacíos; el campo `title` se eliminó por estar vacío en todos los casos.
- El solapamiento léxico medio entre queries y documentos relevantes es de **0,40**.
---
## 2. Carga de datos y preparación

In [1]:
import pandas as pd

# Cargar archivos limpios
corpus = pd.read_csv('../data/subset_corpus_clean.csv')
queries = pd.read_csv('../data/subset_queries_clean.csv')
qrels = pd.read_csv('../data/subset_qrels_clean.csv')

#  estructura y tamaño de cada archivo
print("Corpus:", corpus.shape)
print(corpus.head(2), "\n")

print("Queries:", queries.shape)
print(queries.head(2), "\n")

print("Qrels:", qrels.shape)
print(qrels.head(2))

# Checks
print("\nDuplicados en corpus:", corpus.duplicated().sum())
print("Duplicados en queries:", queries.duplicated().sum())
print("Duplicados en qrels:", qrels.duplicated().sum())

print("\nValores nulos en corpus:\n", corpus.isnull().sum())
print("\nValores nulos en queries:\n", queries.isnull().sum())
print("\nValores nulos en qrels:\n", qrels.isnull().sum())


Corpus: (2998, 4)
    _id                                               text metadata  \
0  1198  Yes, as long as you own the shares before the ...       {}   
1  2003  "While I haven't experienced being ""grad stud...       {}   

   text_length  
0           62  
1           94   

Queries: (300, 4)
    _id                                               text metadata  \
0  3394  What is the easiest way to back-test index fun...       {}   
1  5505  Can I deduct interest and fees on a loan for q...       {}   

   text_length  
0           11  
1           13   

Qrels: (778, 3)
   query_id  doc_id  relevance
0        15  325273          1
1        18   88124          1

Duplicados en corpus: 0
Duplicados en queries: 0
Duplicados en qrels: 0

Valores nulos en corpus:
 _id            0
text           0
metadata       0
text_length    0
dtype: int64

Valores nulos en queries:
 _id            0
text           0
metadata       0
text_length    0
dtype: int64

Valores nulos en qrels:
 query

In [2]:
print("Columnas en corpus:", corpus.columns.tolist())
print("Columnas en queries:", queries.columns.tolist())
print("Columnas en qrels:", qrels.columns.tolist())

print("\nInfo corpus:")
corpus.info()
print("\nInfo queries:")
queries.info()
print("\nInfo qrels:")
qrels.info()

Columnas en corpus: ['_id', 'text', 'metadata', 'text_length']
Columnas en queries: ['_id', 'text', 'metadata', 'text_length']
Columnas en qrels: ['query_id', 'doc_id', 'relevance']

Info corpus:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2998 entries, 0 to 2997
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          2998 non-null   int64 
 1   text         2998 non-null   object
 2   metadata     2998 non-null   object
 3   text_length  2998 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 93.8+ KB

Info queries:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   _id          300 non-null    int64 
 1   text         300 non-null    object
 2   metadata     300 non-null    object
 3   text_length  300 non-null    int64 
dtypes: int64(2), object(2)
memory u

### Estrategias de tokenización evaluadas

Se implementan y comparan varias variantes de tokenización:

- **Básica:** minúsculas + split por espacio (sin eliminar puntuación ni stopwords).
- **Normalización extra:** minúsculas + eliminación de puntuación/símbolos + split.
- **Eliminación de stopwords:** minúsculas + eliminación de puntuación/símbolos + split + eliminación de stopwords (NLTK).
- **Lematización:** minúsculas + eliminación de puntuación/símbolos + split + eliminación de stopwords + lematización (NLTK WordNetLemmatizer).




In [3]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Descargar recursos si no los tienes aún
nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def tokenize(text, mode='basic'):
    # Lowercase
    text = text.lower()
    # Eliminar puntuación y símbolos si corresponde
    if mode in ['nopunct', 'stopwords', 'lemmatize']:
        text = re.sub(r'[^\w\s]', '', text)
    # Split por espacios
    tokens = text.split()
    if mode == 'basic':
        return tokens
    if mode == 'stopwords':
        tokens = [w for w in tokens if w not in stop_words]
    if mode == 'lemmatize':
        tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return tokens


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aalex\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aalex\AppData\Roaming\nltk_data...


In [4]:
from rank_bm25 import BM25Okapi
import numpy as np

def run_bm25_experiment(corpus_df, queries_df, qrels_df, token_mode, result_filename, top_k=10):
    # Tokenizar corpus y queries
    docs = corpus_df['text'].apply(lambda x: tokenize(x, mode=token_mode)).tolist()
    queries = queries_df['text'].apply(lambda x: tokenize(x, mode=token_mode)).tolist()
    doc_ids = corpus_df['_id'].tolist()
    query_ids = queries_df['_id'].tolist()

    bm25 = BM25Okapi(docs)
    results = []
    for q_idx, query_tokens in enumerate(queries):
        scores = bm25.get_scores(query_tokens)
        top_doc_idxs = np.argsort(scores)[::-1][:top_k]
        for rank, idx in enumerate(top_doc_idxs):
            doc_id = doc_ids[idx]
            score = scores[idx]
            is_rel = int(((qrels_df['query_id'] == query_ids[q_idx]) & (qrels_df['doc_id'] == doc_id)).any())
            results.append({
                'query_id': query_ids[q_idx],
                'doc_id': doc_id,
                'score': score,
                'rank': rank+1,
                'is_relevant': is_rel
            })
    # Guardar resultados
    results_df = pd.DataFrame(results)
    results_df.to_csv(f'../results/results_bm25_{token_mode}.csv', index=False)
    print(f"Resultados guardados en ../results/results_bm25_{token_mode}.csv")
    return results_df


In [5]:
def tokenize(text, mode='basic'):
    text = text.lower()
    if mode in ['nopunct', 'stopwords', 'lemmatize']:
        text = re.sub(r'[^\w\s]', '', text)
    tokens = text.split()
    if mode == 'basic':
        return tokens
    if mode == 'nopunct':
        return tokens
    if mode == 'stopwords':
        tokens = [w for w in tokens if w not in stop_words]
    if mode == 'lemmatize':
        tokens = [lemmatizer.lemmatize(w) for w in tokens if w not in stop_words]
    return tokens



----
## 3. Implementación de BM25
Cada archivo contiene, por cada query, los documentos recuperados, su score, el ranking, y la etiqueta de relevancia (`is_relevant`).


In [6]:
for mode in ['basic', 'nopunct', 'stopwords', 'lemmatize']:
    print(f"\n--- Ejecutando BM25 con tokenización: {mode} ---")
    run_bm25_experiment(corpus, queries, qrels, token_mode=mode, result_filename=f'results_bm25_{mode}.csv')



--- Ejecutando BM25 con tokenización: basic ---
Resultados guardados en ../results/results_bm25_basic.csv

--- Ejecutando BM25 con tokenización: nopunct ---
Resultados guardados en ../results/results_bm25_nopunct.csv

--- Ejecutando BM25 con tokenización: stopwords ---
Resultados guardados en ../results/results_bm25_stopwords.csv

--- Ejecutando BM25 con tokenización: lemmatize ---
Resultados guardados en ../results/results_bm25_lemmatize.csv


---
## 4. Evaluación de resultados

En esta sección se evalúan los resultados obtenidos por BM25 bajo las diferentes estrategias de tokenización:

- **Precision@k**: Proporción de documentos relevantes entre los k primeros recuperados para cada query.
- **Recall@k**: Proporción de los documentos relevantes totales que aparecen entre los k primeros recuperados.
- **nDCG@k** (Normalized Discounted Cumulative Gain): Métrica que valora no solo la relevancia, sino la posición en el ranking.
- **MAP** (Mean Average Precision): Promedio de la precisión obtenida en cada punto relevante a lo largo de todas las queries.



In [7]:
import pandas as pd
import numpy as np
from collections import defaultdict

def evaluate_run(results_path, qrels, ks=[1,3,5,10]):
    """
    Evalúa los resultados de un archivo (csv) tipo BM25/SBERT con las métricas estándar.
    - results_path: path al csv con columnas [query_id, doc_id, score, rank, is_relevant]
    - qrels: DataFrame con columnas [query_id, doc_id, relevance]
    """
    results = pd.read_csv(results_path)
    qrels_lookup = defaultdict(set)
    for _, row in qrels.iterrows():
        if row['relevance'] > 0:
            qrels_lookup[row['query_id']].add(row['doc_id'])
    
    metrics = {k: {'precision': [], 'recall': [], 'ndcg': []} for k in ks}
    average_precisions = []

    for qid, group in results.groupby('query_id'):
        retrieved = group.sort_values('rank')['doc_id'].tolist()
        relevant = qrels_lookup[qid]
        binary_relevance = [1 if doc in relevant else 0 for doc in retrieved]

        for k in ks:
            rel_at_k = binary_relevance[:k]
            precision = np.mean(rel_at_k) if rel_at_k else 0.0
            recall = (sum(rel_at_k) / len(relevant)) if relevant else 0.0

            # nDCG@k
            dcg = 0.0
            for i, rel in enumerate(rel_at_k):
                dcg += rel / np.log2(i + 2)
            ideal_rel = [1]*min(len(relevant), k)
            idcg = 0.0
            for i, rel in enumerate(ideal_rel):
                idcg += rel / np.log2(i + 2)
            ndcg = dcg/idcg if idcg > 0 else 0.0

            metrics[k]['precision'].append(precision)
            metrics[k]['recall'].append(recall)
            metrics[k]['ndcg'].append(ndcg)
        
        # AP (Average Precision)
        n_rels = 0
        ap = 0.0
        for i, rel in enumerate(binary_relevance, 1):
            if rel:
                n_rels += 1
                ap += n_rels / i
        ap = ap / len(relevant) if relevant else 0.0
        average_precisions.append(ap)

    results_summary = {}
    for k in ks:
        results_summary[f'Precision@{k}'] = np.mean(metrics[k]['precision'])
        results_summary[f'Recall@{k}'] = np.mean(metrics[k]['recall'])
        results_summary[f'nDCG@{k}'] = np.mean(metrics[k]['ndcg'])
    results_summary['MAP'] = np.mean(average_precisions)
    return results_summary

# ---- Evaluar todas las variantes BM25 ----
qrels_clean = pd.read_csv('../data/subset_qrels_clean.csv')
bm25_variants = ['basic', 'nopunct', 'stopwords', 'lemmatize']
summary_table = []

for mode in bm25_variants:
    print(f"Evaluando BM25 ({mode})...")
    summary = evaluate_run(f'../results/results_bm25_{mode}.csv', qrels_clean, ks=[1,3,5,10])
    summary['Variante'] = mode
    summary_table.append(summary)

summary_df = pd.DataFrame(summary_table).set_index('Variante')
display(summary_df)


Evaluando BM25 (basic)...
Evaluando BM25 (nopunct)...
Evaluando BM25 (stopwords)...
Evaluando BM25 (lemmatize)...


,Precision@1,Recall@1,nDCG@1,Precision@3,Recall@3,nDCG@3,Precision@5,Recall@5,nDCG@5,Precision@10,Recall@10,nDCG@10,MAP
Variante,,,,,,,,,,,,,
basic,0.340000,0.163998,0.340000,0.204444,0.275367,0.308292,0.147333,0.314999,0.311860,0.093000,0.384978,0.333482,0.261893
nopunct,0.423333,0.213543,0.423333,0.267778,0.362251,0.400553,0.197333,0.422902,0.412140,0.125333,0.521240,0.442866,0.357781
stopwords,0.446667,0.230364,0.446667,0.284444,0.389593,0.423066,0.209333,0.458282,0.438351,0.127333,0.536131,0.461636,0.378523
lemmatize,0.450000,0.236123,0.450000,0.288889,0.412816,0.440240,0.214000,0.483444,0.456962,0.130333,0.552623,0.478584,0.396863


### Resultados de la variante "lemmatize"

Para la variante lematizada, los resultados por distintos valores de k son los siguientes:

- **Precision@1:** 0.450
- **Precision@3:** 0.289
- **Precision@10:** 0.130

El valor medio de documentos relevantes por query en el subset es **2.59**. Esto significa que, aunque solo hay un promedio de 2 o 3 documentos relevantes por pregunta, el sistema logra encontrar uno relevante en la primera posición en el 45% de los casos, y aumenta la probabilidad a medida que se amplía k, aunque la precisión baja como es esperable.

- **Recall@1:** 0.236
- **Recall@3:** 0.413
- **Recall@10:** 0.553

El recall, logicamente, sube con k, indicando que si se consideran más resultados (top-10), el sistema es capaz de recuperar una mayor parte de los documentos relevantes para cada query. Buena precisión en el top-1 y mejora de recall al ampliar el número de documentos recuperados.


---
## 5. Análisis cualitativo

In [10]:
import pandas as pd

# Cargar resultados de BM25 lematizado y qrels
results = pd.read_csv('../results/results_bm25_lemmatize.csv')
qrels = pd.read_csv('../data/subset_qrels_clean.csv')
queries = pd.read_csv('../data/subset_queries_clean.csv')
corpus = pd.read_csv('../data/subset_corpus_clean.csv')

# Asegúrate de que los tipos de ID son consistentes
results['query_id'] = results['query_id'].astype(int)
results['doc_id'] = results['doc_id'].astype(int)
qrels['query_id'] = qrels['query_id'].astype(int)
qrels['doc_id'] = qrels['doc_id'].astype(int)

# Para cada query: ¿el relevante está en rank 1? (fácil) ¿ningún relevante en top-10? (difícil)
def get_easy_hard_queries(results, qrels, top_k=10):
    easy = []
    hard = []
    grouped = results.groupby('query_id')
    for qid, group in grouped:
        # docs recuperados y su relevancia (ordenados por rank)
        topk = group.sort_values('rank').head(top_k)
        relevant_docs = set(qrels[qrels['query_id'] == qid]['doc_id'])
        # ¿Algún relevante en rank 1?
        if topk.iloc[0]['is_relevant'] == 1:
            easy.append(qid)
        # ¿Ningún relevante en top-10?
        if not any([doc in relevant_docs for doc in topk['doc_id']]):
            hard.append(qid)
    return easy, hard

easy_qids, hard_qids = get_easy_hard_queries(results, qrels, top_k=10)

# Mostrar ejemplos de queries fáciles
print("\nEJEMPLOS DE QUERIES FÁCILES (relevante en posición 1):\n")
for qid in easy_qids[:3]:  # muestra los 3 primeros como ejemplo
    q_text = queries.loc[queries['_id'] == qid, 'text'].values[0]
    print(f"Query: {q_text}")
    top1_doc_id = results[(results['query_id'] == qid) & (results['rank'] == 1)]['doc_id'].values[0]
    doc_text = corpus.loc[corpus['_id'] == top1_doc_id, 'text'].values[0]
    print(f"Doc recuperado (top-1): {doc_text[:200]}...\n")

# Mostrar ejemplos de queries difíciles
print("\nEJEMPLOS DE QUERIES DIFÍCILES (ningún relevante en top-10):\n")
for qid in hard_qids[:3]:  # muestra los 3 primeros como ejemplo
    q_text = queries.loc[queries['_id'] == qid, 'text'].values[0]
    print(f"Query: {q_text}")
    # Mostrar top-1 recuperado
    top1_doc_id = results[(results['query_id'] == qid) & (results['rank'] == 1)]['doc_id'].values[0]
    doc_text = corpus.loc[corpus['_id'] == top1_doc_id, 'text'].values[0]
    print(f"Doc recuperado (top-1): {doc_text[:200]}...")
    print(f"(Ningún documento relevante recuperado en el top-10)\n")



EJEMPLOS DE QUERIES FÁCILES (relevante en posición 1):

Query: How to account for money earned and spent prior to establishing business bank accounts?
Doc recuperado (top-1): Funds earned and spent before opening a dedicated business account should be classified according to their origination. For example, if your business received income, where did that money go?  If you ...

Query: financial institution wants share member break down for single member LLC
Doc recuperado (top-1): "What exactly would the financial institution need to see to make them   comfortable with these regulations The LLC Operating Agreement. The OA should specify the member's allocation of equity, assets...

Query: Where to request ACH Direct DEBIT of funds from MY OWN personal bank account?
Doc recuperado (top-1): Call Wells Fargo or go to a branch.  Tell them what you're trying to accomplish, not the vehicle you think you should use to get there.  Don't tell them you want to ACH DEBIT from YOUR ACCOUNT of YOUR

#### Análisis cualitativo

BM25 con lematización acierta cuando la query y el documento comparten palabras, si no hay coincidencia clara de términos, el sistema falla y no recupera los relevantes en el top-10.

Este comportamiento es esperable en BM25, sobre todo cuando las queries y los documentos son cortos.  
Sirve como baseline, pero tiene limitaciones.


---

In [9]:
import pickle
from rank_bm25 import BM25Okapi

bm25_variants = ['basic', 'nopunct', 'stopwords', 'lemmatize']

for token_mode in bm25_variants:
    print(f"Guardando BM25 ({token_mode})...")
    docs_tokenized = corpus['text'].apply(lambda x: tokenize(x, mode=token_mode)).tolist()
    bm25 = BM25Okapi(docs_tokenized)
    with open(f'../models/bm25_{token_mode}.pkl', 'wb') as f:
        pickle.dump(bm25, f)
    print(f"Guardado en ../models/bm25_{token_mode}.pkl")


Guardando BM25 (basic)...
Guardado en ../models/bm25_basic.pkl
Guardando BM25 (nopunct)...
Guardado en ../models/bm25_nopunct.pkl
Guardando BM25 (stopwords)...
Guardado en ../models/bm25_stopwords.pkl
Guardando BM25 (lemmatize)...
Guardado en ../models/bm25_lemmatize.pkl
